# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [9]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

# Preprocesamiento de datos
spaceship['CryoSleep'].fillna(False, inplace=True)
spaceship['Age'].fillna(spaceship['Age'].median(), inplace=True)
spaceship['RoomService'].fillna(0, inplace=True)
spaceship['FoodCourt'].fillna(0, inplace=True)
spaceship['ShoppingMall'].fillna(0, inplace=True)
spaceship['Spa'].fillna(0, inplace=True)
spaceship['VRDeck'].fillna(0, inplace=True)
spaceship['HomePlanet'].fillna('Earth', inplace=True)
spaceship['Destination'].fillna('TRAPPIST-1e', inplace=True)
spaceship['VIP'].fillna(False, inplace=True)

# Convertir variables categóricas a variables dummy
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet', 'Destination', 'Cabin'], drop_first=True)

# Separar características y objetivo
X = spaceship.drop(columns=['PassengerId', 'Name', 'Transported'])
y = spaceship['Transported']

# Escalado de características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Selección de características
selector = SelectKBest(score_func=f_classif, k=10)
X_selected = selector.fit_transform(X_scaled, y)

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Mostrar las características seleccionadas
selected_features = X.columns[selector.get_support()]
print("Características seleccionadas:", selected_features)

/tmp/ipykernel_132181/1477220780.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  spaceship['CryoSleep'].fillna(False, inplace=True)
/tmp/ipykernel_132181/1477220780.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spaceship['CryoSleep'].fillna(False, inplace=True)
/tmp/ipykernel_132181/1477220780.py:6: F

Características seleccionadas: Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'Spa', 'VRDeck',
       'HomePlanet_Europa', 'Destination_TRAPPIST-1e', 'Cabin_G/981/S'],
      dtype='object')


**Perform Train Test Split**

In [10]:
# Perform the train-test split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [16]:
# Bagging and Pasting
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Bagging
bagging = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
accuracy_bagging = accuracy_score(y_test, y_pred)
print("Bagging Accuracy:", accuracy_bagging)

# Pasting
pasting = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42, bootstrap=False)
pasting.fit(X_train, y_train)
y_pred = pasting.predict(X_test)
accuracy_pasting = accuracy_score(y_test, y_pred)
print("Pasting Accuracy:", accuracy_pasting)


Bagging Accuracy: 0.7694077055779184
Pasting Accuracy: 0.7239792984473835


- Random Forests

In [17]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

# Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
accuracy_random_forest = accuracy_score(y_test, y_pred)
print("Random Forest Accuracy:", accuracy_random_forest)



Random Forest Accuracy: 0.772857964347326


- Gradient Boosting

In [18]:
# Gradient Boosting

from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting
gradient_boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)
gradient_boosting.fit(X_train, y_train)
y_pred = gradient_boosting.predict(X_test)
accuracy_gradient_boosting = accuracy_score(y_test, y_pred)
print("Gradient Boosting Accuracy:", accuracy_gradient_boosting)


Gradient Boosting Accuracy: 0.777458309373203


- Adaptive Boosting

In [19]:
# AdaBoost

from sklearn.ensemble import AdaBoostClassifier

# AdaBoost
adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)
accuracy_adaboost = accuracy_score(y_test, y_pred)
print("AdaBoost Accuracy:", accuracy_adaboost)


/home/benjamin/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.7797584818861415


Which model is the best and why?

In [21]:
print("Bagging Accuracy:", accuracy_bagging)
print("Pasting Accuracy:", accuracy_pasting)
print("Random Forest Accuracy:", accuracy_random_forest)
print("Gradient Boosting Accuracy:", accuracy_gradient_boosting)
print("AdaBoost Accuracy:", accuracy_adaboost)

Bagging Accuracy: 0.7694077055779184
Pasting Accuracy: 0.7239792984473835
Random Forest Accuracy: 0.772857964347326
Gradient Boosting Accuracy: 0.777458309373203
AdaBoost Accuracy: 0.7797584818861415


In [ ]:
#comment here

# The best model is AdaBoost with an accuracy of 0.779